In [ ]:
import numpy as np
from lxml import objectify

import mbuild as mb
from mbuild.lib.recipes import Polymer
import foyer
import ele

import foyer_xml_writer
from foyer_xml_writer import parmed_to_foyer_xml, 
from foyer_xml_writer import *

import warnings
warnings.filterwarnings("ignore")

# Example:

### Create an arbitrary foyer xml file we can use with a abstract system

#### Workflow:
1. Create molecule or system in mBuild
2. Create data structure containing our abstract parameters
3. Write out a useable Foyer forcefield file

In [ ]:
mer = mb.Compound()
mer.add(mb.Compound(name="A", pos=[0,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[0.5,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[1.0,0,0], mass=1))
mer.freud_generate_bonds(name_a="A", name_b="A", dmax=0.51, dmin=0.049)

chain = Polymer()
chain.add_monomer(mer, indices=[0, 2], replace=False, orientation=[[-1, 0, 0], [1,0,0]], separation=0.5)
chain.build(n=3, add_hydrogens=False)
chain.visualize().show()

box_of_chains = mb.fill_box(compound=chain, n_compounds=10, box=[10, 10, 10])
box_of_chains.visualize().show()

In [ ]:
bond_parameters = {
    ("A", "A"): {"k": 50, "l0": 0.5}
}

angle_parameters = {
    ("A", "A", "A"): {"k": 5, "t0": 2.2}
}

dihedral_parameters = {
    ("A", "A", "A", "A"): {
        "periodicity": [2],
        "k": [10],
        "phase": [3.14]
    }
}

non_bonded_parameters = {
    "A": {"sigma": 1.0, "epsilon": 1.0, "charge": 0}
}

In [ ]:
def mbuild_to_foyer_xml(
    file_name=None,
    compound=None,
    bond_params=None,
    angle_params=None,
    dihedral_params=None,
    dihedral_type="periodic",
    non_bonded_params=None,
    combining_rule=None,
    name="",
    version="",
    coulomb14scale="",
    lj14scale=""
):
    
    particle_types = tuple(set(p.name for p in compound.particles()))
    particle_masses = []
    for _type in particle_types:
        mass = [p.mass for p in compound.particles_by_name(_type)][0]
        particle_masses.append(mass)
    
    
    print(particle_types)
    with open(file_name, "w") as f:
        f.write(f'<ForceField name="{name}" version="{version}" combining_rule="{combining_rule}">\n')
        f.write("\t<AtomTypes>\n")
        for idx, p in enumerate(particle_types):
            line = write_atom_type(
                name=p,
                atom_type=p,
                element=f"_{p}",
                mass=particle_masses[idx],
                _def=f"_{p}",
            )
            f.write(line)
        f.write("\t</AtomTypes>\n")
        
        f.write("<HarmonicBondForce>\n")
        for b in bond_params:
            line = write_harmonic_bond(
                class1=b[0],
                class2=b[1],
                l0=bond_params[b]["l0"],
                k=bond_params[b]["k"]
            )
            f.write(line)
        f.write("</HarmonicBondForce>\n")
        
        f.write("<HarmonicAngleForce>\n")
        for a in angle_params:
            line = write_harmonic_angle(
                class1=a[0],
                class2=a[1],
                class3=a[2],
                t0=angle_params[a]["t0"],
                k=angle_params[a]["k"]
            )
            f.write(line)
        f.write("</HarmonicAngleForce>\n")
        
        if dihedral_type == "periodic":
            f.write("<PeriodicTorsionForce>\n")
            for d in dihedral_params:
                line = write_periodic_dihedral(
                    class1=d[0],
                    class2=d[1],
                    class3=d[2],
                    class4=d[3],
                    periodicity=dihedral_params[d]["periodicity"],
                    k=dihedral_params[d]["k"],
                    phase=dihedral_params[d]["phase"],
                )
                f.write(line)
            f.write("</PeriodicTorsionForce>\n")
        
        f.write(f'\t<NonbondedForce coulomb14scale="{coulomb14scale}" lj14scale="{lj14scale}">\n')
        for a in non_bonded_params:
            line = write_non_bonded(
                name=a,
                charge=non_bonded_params[a]["charge"],
                sigma=non_bonded_params[a]["sigma"],
                epsilon=non_bonded_params[a]["epsilon"]
            )
            f.write(line)
        f.write('\t</NonbondedForce>\n')
        f.write('</ForceField>')

    for p in compound.particles():
        p.name = f"_{p.name}"

In [ ]:
mbuild_to_foyer_xml(
    file_name="testmb.xml",
    compound=box_of_chains,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    non_bonded_params=non_bonded_parameters
)

In [ ]:
mylist = ["A","A","A","A"]

In [ ]:
mylist.extend([""]*(4-len(mylist)))

In [ ]:
mylist

# Example:

### Write out a trimmed-down forcefield file that only has the parameters we're using.

#### This works with our typical workflow:

1. Create molecule or system in mBuild
2. Use an existing focefield along with Foyer to apply the forcefield
3. Write out a useable, trimmed down foyer forcefield file

In [ ]:
# mBuild system
alkane = mb.load("CCCC", smiles=True)
alkane_box = mb.fill_box(alkane, n_compounds=10, box=[2, 2, 2])

# opls forcefield form foyer
ff = foyer.forcefields.get_forcefield(name="opls")

# parmed structure
alkane_pmd = ff.apply(alkane_box, assert_dihedral_params=True)

In [ ]:
# Save the trimmed xml file:
parmed_to_foyer_xml(structure=alkane_pmd, ff=ff, file_name="alkane_opls.xml")

In [ ]:
# Test out the xml file we created; we should be able to use it directly with Foyer:
ff_test = foyer.Forcefield(forcefield_files="alkane_opls.xml")
alkane_pmd_test = ff_test.apply(alkane_box)

In [ ]:
def write_atom_type(name, atom_type, element, mass, _def=None, desc=None):
    line = f'\t\t<Type name="{name}" class="{atom_type}" element="{element}" mass="{mass}" def="{_def}" desc="{desc}"/>\n'
    return line


def write_harmonic_bond(class1, class2, l0, k):
    line = f'\t\t<Bond class1="{class1}" class2="{class2}" length="{l0}" k="{k}"/>\n'
    return line


def write_harmonic_angle(class1, class2, class3, t0, k):
    line = f'\t\t<Angle class1="{class1}" class2="{class2}" class3="{class3}" angle="{t0}" k="{k}"/>\n'
    return line


def write_non_bonded(name, charge, sigma, epsilon):
    line = f'\t\t<Atom type="{name}" charge="{charge}" sigma="{sigma}" epsilon="{epsilon}"/>\n'
    return line


def write_rb_torsion(class1, class2, class3, class4, c0, c1, c2, c3, c4, c5):
    line = f'\t\t<Proper class1="{class1}" class2="{class2}" class3="{class3}" class4="{class4}" c0="{c0}" c1="{c1}" c2="{c2}" c3="{c3}" c4="{c4}" c5="{c5}"/>\n'
    return line


def write_dihedral():
    pass


def get_atom_types(structure):
    return tuple(set(a.type for a in structure.atoms))


def get_bond_types(structure):
    bond_types = set()
    for i in structure.bonds:
        bond_types.add((i.atom1.type, i.atom2.type))
    return tuple(bond_types)


def get_angle_types(structure):
    angle_types = set()
    for i in structure.angles:
        angle_types.add((i.atom1.type, i.atom2.type, i.atom3.type))
    return tuple(angle_types)


def get_dihedral_types(structure):
    """Use this if the parmed structure was created using the GAFF forcefield"""
    dihedral_types = set()
    for i in structure.dihedrals:
        dihedral_types.add(
            (i.atom1.type, i.atom2.type, i.atom3.type, i.atom4.type)
        )
    return tuple(dihedral_types)


def get_rb_tortions(structure):
    """Use this if the parmed structure was created using the OPLS forcefield"""
    dihedral_types = set()
    for i in structure.rb_torsions:
        dihedral_types.add(
            (i.atom1.type, i.atom2.type, i.atom3.type, i.atom4.type)
        )
    return tuple(dihedral_types)

In [ ]:
def parmed_to_foyer_xml(structure, ff, file_name):
    with open(file_name, "w") as f:
        f.write(f'<ForceField name="{ff.name}" version="{ff.version}" combining_rule="{ff.combining_rule}">\n')
        f.write("\t<AtomTypes>\n")
        for atom in get_atom_types(structure):
            atom_type = ff.atomTypeClasses[atom]
            element=ff.atomTypeElements[atom]
            mass = ele.element_from_symbol(element).mass
            _def = ff.atomTypeDefinitions[atom]
            desc = ff.atomTypeDesc[atom]
            line = write_atom_type(
                name=atom,
                atom_type=ff.atomTypeClasses[atom],
                element=ff.atomTypeElements[atom],
                mass=mass,
                _def=_def,
                desc=desc
            )
            f.write(line)
        f.write("\t</AtomTypes>\n")
        
        f.write("\t<HarmonicBondForce>\n")
        for bond in get_bond_types(structure):
            params = ff.get_parameters("harmonic_bonds", bond)
            class1 = ff.atomTypeClasses[bond[0]]
            class2 = ff.atomTypeClasses[bond[1]]
            line = write_harmonic_bond(class1=class1, class2=class2, l0=params["length"], k=params["k"])
            f.write(line)
        f.write("\t</HarmonicBondForce>\n")
        
        f.write("\t<HarmonicAngleForce>\n")
        for angle in get_angle_types(structure):
            params=ff.get_parameters("harmonic_angles", angle)
            class1 = ff.atomTypeClasses[angle[0]]
            class2 = ff.atomTypeClasses[angle[1]]
            class3 = ff.atomTypeClasses[angle[2]]
            line = write_harmonic_angle(
                class1=class1, class2=class2, class3=class3, t0=params["theta"], k=params["k"]
            )
            f.write(line)
        f.write("\t</HarmonicAngleForce>\n")
        
        if "OPLS" in ff.name:
            f.write("\t<RBTorsionForce>\n")
            for dihedral in get_rb_tortions(structure):
                params = ff.get_parameters("rb_propers", dihedral)
                class1 = ff.atomTypeClasses[dihedral[0]]
                class2 = ff.atomTypeClasses[dihedral[1]]
                class3 = ff.atomTypeClasses[dihedral[2]]
                class4 = ff.atomTypeClasses[dihedral[3]]
                line = write_rb_torsion(
                    class1=class1,
                    class2=class2,
                    class3=class3,
                    class4=class4,
                    c0=params["c0"],
                    c1=params["c1"],
                    c2=params["c2"],
                    c3=params["c3"],
                    c4=params["c4"],
                    c5=params["c5"],
                )
                f.write(line)
            f.write("\t</RBTorsionForce>\n")
        
        f.write(f'\t<NonbondedForce coulomb14scale="{ff.coulomb14scale}" lj14scale="{ff.lj14scale}">\n')
        for atom in get_atom_types(structure):
            params = ff.get_parameters("atoms", atom)
            line = write_non_bonded(
                name=atom, charge=params["charge"], sigma=params["sigma"], epsilon=params["epsilon"]
            )
            f.write(line)
        f.write('\t</NonbondedForce>\n')
        f.write('</ForceField>')

In [ ]:
parmed_to_foyer_xml(methane_pmd, ff=ff, file_name="testing.xml")

In [ ]:
ff_test = foyer.Forcefield(forcefield_files="test.xml")

In [ ]:
methane_pmd2 = ff_test.apply(methane)